In [37]:
import cv2
import numpy as np
import tensorflow as tf
import os
from random import shuffle
from tqdm import tqdm

In [38]:
TRAIN_DIR = '/home/clean/Documents/Dog vs Cat project/train'
TEST_DIR = '/home/clean/Documents/Dog vs Cat project/test'
IMG_SIZE = 50
LR = 0.001

MODE_NAME = 'dogvscats-{}-{}.model'.format(LR, '2conv-basic')

In [39]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat' : return [1,0]
    elif word_label == 'dog' : return [0,1]

In [47]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data
        
        

In [48]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
        np.save('test_data.npy', testing_data)
        return testing_data

In [49]:
train_data = create_train_data()

100%|██████████| 25000/25000 [05:53<00:00, 70.77it/s] 


In [50]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tflearn.datasets.mnist as mnist

X, Y, test_x, test_y = mnist.load_data(one_hot=True)

X = X.reshape([-1, 28, 28, 1])
test_x = test_x.reshape([-1, 28, 28, 1])

convnet = input_data(shape=[None, 28, 28, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 10, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet)
model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id='mnist')

Training Step: 8599  | total loss: 0.04677 | time: 77.795s
| Adam | epoch: 010 | loss: 0.04677 - acc: 0.9875 -- iter: 54976/55000
Training Step: 8600  | total loss: 0.04245 | time: 80.524s
| Adam | epoch: 010 | loss: 0.04245 - acc: 0.9887 | val_loss: 0.06950 - val_acc: 0.9830 -- iter: 55000/55000
--
